<a href="https://colab.research.google.com/github/Digitalinnstudio/data_cleaning_python/blob/main/sales_dataset_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
df=pd.read_excel('/content/dim_sales.xlsx')

In [4]:
df.head()

,Order ID,Customer Name,Order Date,Ship Date,Product,Quantity,Price,Total Price,Region
0,2001,Chris Paul,2025-03-08,2025-03-28,Charger,3,50,-85,East
1,2002,Eva White,03/07/2025,2025-03-28,Tablet,2,$600,1525,East
2,2003,Olivia Adams,03/07/2025,2025-03-28,Monitor,2,50,4007,North
3,2004,Robert Finch,03/14/2025,2025-03-28,Mouse,-2,650,1848,West
4,2005,Sarah Lee,03/01/2025,2025-03-28,Monitor,1,$300,1895,North


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       400 non-null    int64         
 1   Customer Name  370 non-null    object        
 2   Order Date     400 non-null    object        
 3   Ship Date      400 non-null    datetime64[ns]
 4   Product        400 non-null    object        
 5   Quantity       400 non-null    object        
 6   Price          400 non-null    object        
 7   Total Price    400 non-null    int64         
 8   Region         346 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 28.3+ KB


In [7]:
df['Price'] = df['Price'].astype(str).str.replace(r'[£,]', '', regex=True)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

print("Data types after converting 'Price' and 'Quantity' columns:")
print(df[['Price', 'Quantity']].dtypes)

Data types after converting 'Price' and 'Quantity' columns:
Price       float64
Quantity    float64
dtype: object


In [9]:
print("\nDescriptive statistics for numerical columns:")
print(df.describe())

categorical_cols = ['Product', 'Region', 'Customer Name']

print("\nAnalysis of categorical columns:")
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(f"Number of unique values: {df[col].nunique()}")
    print(f"Top 5 most frequent values:\n{df[col].value_counts().head()}")


Descriptive statistics for numerical columns:
          Order ID            Ship Date    Quantity       Price  Total Price
count   400.000000                  400  310.000000  196.000000   400.000000
mean   2200.500000  2025-03-28 00:00:00    1.383871  348.852041  2451.667500
min    2001.000000  2025-03-28 00:00:00   -3.000000   50.000000 -4377.000000
25%    2100.750000  2025-03-28 00:00:00   -2.000000   75.000000  1181.000000
50%    2200.500000  2025-03-28 00:00:00    2.000000  100.000000  2541.500000
75%    2300.250000  2025-03-28 00:00:00    3.750000  650.000000  3783.500000
max    2400.000000  2025-03-28 00:00:00    5.000000  850.000000  4991.000000
std     115.614301                  NaN    2.742049  336.357690  1599.791362

Analysis of categorical columns:

Column: Product
Number of unique values: 8
Top 5 most frequent values:
Product
Mouse       62
Laptop      54
Tablet      53
Monitor     51
Keyboard    51
Name: count, dtype: int64

Column: Region
Number of unique values: 4
To

In [10]:
missing_values = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_info = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
})

missing_info = missing_info.sort_values(by='Missing Percentage', ascending=False)

print("\nMissing Values Information:")
print(missing_info[missing_info['Missing Count'] > 0])


Missing Values Information:
               Missing Count  Missing Percentage
Price                    204                51.0
Quantity                  90                22.5
Region                    54                13.5
Customer Name             30                 7.5


In [11]:
duplicate_rows_count = df.duplicated().sum()
print(f"Total number of duplicate rows: {duplicate_rows_count}")

Total number of duplicate rows: 0


In [12]:
missing_or_negative_price_mask = (df['Price'].isnull()) | (df['Price'] < 0)
non_zero_quantity_mask = (df['Quantity'] != 0) & (df['Quantity'].notnull())

# Combine masks to find rows where we can attempt to calculate price
calculable_price_mask = missing_or_negative_price_mask & non_zero_quantity_mask

df['calculated_price'] = np.nan
df.loc[calculable_price_mask, 'calculated_price'] = df.loc[calculable_price_mask, 'Total Price'] / df.loc[calculable_price_mask, 'Quantity']

print(f"Number of rows where 'Price' was missing/negative and 'Quantity' was non-zero for calculation: {calculable_price_mask.sum()}")
print("First 5 rows with calculated price (if any):")
print(df[calculable_price_mask][['Price', 'Quantity', 'Total Price', 'calculated_price']].head())

Number of rows where 'Price' was missing/negative and 'Quantity' was non-zero for calculation: 166
First 5 rows with calculated price (if any):
   Price  Quantity  Total Price  calculated_price
1    NaN       2.0         1525             762.5
4    NaN       1.0         1895            1895.0
5    NaN       2.0         1458             729.0
7    NaN       3.0         2988             996.0
8    NaN       4.0         1350             337.5


In [13]:
df.loc[calculable_price_mask, 'calculated_price'] = np.abs(df.loc[calculable_price_mask, 'Total Price'] / df.loc[calculable_price_mask, 'Quantity'])

df.loc[missing_or_negative_price_mask, 'Price'] = df.loc[missing_or_negative_price_mask, 'calculated_price']

df['Quantity'] = df['Quantity'].abs()

df['Total Price'] = df['Price'] * df['Quantity']

print("Price, Quantity, and Total Price after imputation and cleaning negative quantities:")
print(df[['Price', 'Quantity', 'Total Price']].describe())

df.drop(columns=['calculated_price'], inplace=True)

Price, Quantity, and Total Price after imputation and cleaning negative quantities:
             Price    Quantity  Total Price
count   362.000000  310.000000   310.000000
mean    735.378637    2.809677  1905.161290
std     823.842814    1.233002  1545.567645
min      19.666667    1.000000    50.000000
25%     100.000000    2.000000   345.000000
50%     650.000000    3.000000  1700.000000
75%     854.900000    3.750000  3224.250000
max    4990.000000    5.000000  4990.000000


In [14]:
df.loc[calculable_price_mask, 'calculated_price'] = np.abs(df.loc[calculable_price_mask, 'Total Price'] / df.loc[calculable_price_mask, 'Quantity'])

df.loc[missing_or_negative_price_mask, 'Price'] = df.loc[missing_or_negative_price_mask, 'calculated_price']

df['Quantity'] = df['Quantity'].abs()

df['Total Price'] = df['Price'] * df['Quantity']

print("Price, Quantity, and Total Price after imputation and cleaning negative quantities:")
print(df[['Price', 'Quantity', 'Total Price']].describe())

df.drop(columns=['calculated_price'], inplace=True)


Price, Quantity, and Total Price after imputation and cleaning negative quantities:
             Price    Quantity  Total Price
count   362.000000  310.000000   310.000000
mean    735.378637    2.809677  1905.161290
std     823.842814    1.233002  1545.567645
min      19.666667    1.000000    50.000000
25%     100.000000    2.000000   345.000000
50%     650.000000    3.000000  1700.000000
75%     854.900000    3.750000  3224.250000
max    4990.000000    5.000000  4990.000000


In [15]:
df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

nat_count = df['Order Date'].isnull().sum()

print(f"Number of NaT values in 'Order Date' after conversion: {nat_count}")
print("Data type of 'Order Date' after conversion:")
print(df['Order Date'].dtype)

Number of NaT values in 'Order Date' after conversion: 288
Data type of 'Order Date' after conversion:
datetime64[ns]


In [16]:
median_quantity = df['Quantity'].median()
df['Quantity'].fillna(median_quantity, inplace=True)

print(f"Missing values in 'Quantity' after imputation: {df['Quantity'].isnull().sum()}")

df['Quantity'] = df['Quantity'].astype(int)

print("Data type of 'Quantity' after conversion:")
print(df['Quantity'].dtype)

Missing values in 'Quantity' after imputation: 0
Data type of 'Quantity' after conversion:
int64


/tmp/ipython-input-3169563748.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Quantity'].fillna(median_quantity, inplace=True)


In [17]:
median_quantity = df['Quantity'].median()
df['Quantity'] = df['Quantity'].fillna(median_quantity)

print(f"Missing values in 'Quantity' after imputation: {df['Quantity'].isnull().sum()}")

df['Quantity'] = df['Quantity'].astype(int)

print("Data type of 'Quantity' after conversion:")
print(df['Quantity'].dtype)

Missing values in 'Quantity' after imputation: 0
Data type of 'Quantity' after conversion:
int64


In [18]:
missing_price_mask = df['Price'].isnull()
unresolvable_price_df = df[missing_price_mask].copy()

print("First 5 rows of unresolvable_price_df (rows with missing 'Price'):")
print(unresolvable_price_df.head())

First 5 rows of unresolvable_price_df (rows with missing 'Price'):
    Order ID Customer Name Order Date  Ship Date     Product  Quantity  Price  \
23      2024    Lucas Gray        NaT 2025-03-28      Laptop         3    NaN   
26      2027    Jane Smith        NaT 2025-03-28      Tablet         3    NaN   
35      2036   David Black        NaT 2025-03-28  Headphones         3    NaN   
68      2069     Eva White        NaT 2025-03-28    Keyboard         3    NaN   
74      2075     Sarah Lee        NaT 2025-03-28     Monitor         3    NaN   

    Total Price Region  
23          NaN  South  
26          NaN  North  
35          NaN  South  
68          NaN  South  
74          NaN  North  


In [19]:
df = df[~missing_price_mask].copy()

print(f"Number of rows remaining in df after removing unresolvable prices: {len(df)}")
print(f"Number of rows in unresolvable_price_df: {len(unresolvable_price_df)}")

Number of rows remaining in df after removing unresolvable prices: 362
Number of rows in unresolvable_price_df: 38


In [20]:
unresolvable_price_df.to_csv('unresolvable_prices.csv', index=False)
print("unresolvable_price_df exported to 'unresolvable_prices.csv'")

unresolvable_price_df exported to 'unresolvable_prices.csv'


In [21]:
missing_region = df['Region'].isnull().sum()
missing_customer_name = df['Customer Name'].isnull().sum()

print(f"Missing values in 'Region': {missing_region}")
print(f"Missing values in 'Customer Name': {missing_customer_name}")

Missing values in 'Region': 49
Missing values in 'Customer Name': 25


In [22]:
df['Region'].fillna('Unknown', inplace=True)
df['Customer Name'].fillna('Unknown', inplace=True)

print(f"Missing values in 'Region' after imputation: {df['Region'].isnull().sum()}")
print(f"Missing values in 'Customer Name' after imputation: {df['Customer Name'].isnull().sum()}")

Missing values in 'Region' after imputation: 0
Missing values in 'Customer Name' after imputation: 0


/tmp/ipython-input-3497945121.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Region'].fillna('Unknown', inplace=True)
/tmp/ipython-input-3497945121.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [24]:
df['Region'] = df['Region'].fillna('Unknown')
df['Customer Name'] = df['Customer Name'].fillna('Unknown')

print(f"Missing values in 'Region' after imputation: {df['Region'].isnull().sum()}")
print(f"Missing values in 'Customer Name' after imputation: {df['Customer Name'].isnull().sum()}")

Missing values in 'Region' after imputation: 0
Missing values in 'Customer Name' after imputation: 0


In [25]:
print("\nDataFrame Info:")
df.info()

print("\nDescriptive Statistics:")
print(df.describe())

print("\nMissing Values after cleaning:")
print(df.isnull().sum())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 0 to 399
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       362 non-null    int64         
 1   Customer Name  362 non-null    object        
 2   Order Date     104 non-null    datetime64[ns]
 3   Ship Date      362 non-null    datetime64[ns]
 4   Product        362 non-null    object        
 5   Quantity       362 non-null    int64         
 6   Price          362 non-null    float64       
 7   Total Price    310 non-null    float64       
 8   Region         362 non-null    object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 28.3+ KB

Descriptive Statistics:
          Order ID                     Order Date            Ship Date  \
count   362.000000                            104                  362   
mean   2198.801105  2025-03-10 06:13:50.769230848  2025-03-28 00:00:

In [27]:
df['instore_purchase'] = 'N'

instore_mask = (df['Order Date'].isnull()) | (df['Order Date'] == df['Ship Date'])
df.loc[instore_mask, 'instore_purchase'] = 'Y'

print("Value counts for 'instore_purchase' column:")
print(df['instore_purchase'].value_counts())

print("\nFirst 10 rows with 'instore_purchase' column:")
print(df[['Order Date', 'Ship Date', 'instore_purchase']].head(10))

Value counts for 'instore_purchase' column:
instore_purchase
Y    258
N    104
Name: count, dtype: int64

First 10 rows with 'instore_purchase' column:
  Order Date  Ship Date instore_purchase
0 2025-03-08 2025-03-28                N
1        NaT 2025-03-28                Y
2        NaT 2025-03-28                Y
3        NaT 2025-03-28                Y
4        NaT 2025-03-28                Y
5        NaT 2025-03-28                Y
6        NaT 2025-03-28                Y
7        NaT 2025-03-28                Y
8        NaT 2025-03-28                Y
9        NaT 2025-03-28                Y


In [29]:
nat_order_date_mask = df['Order Date'].isnull()

temp_ship_date = df.loc[nat_order_date_mask, 'Ship Date']


df.loc[nat_order_date_mask, 'Order Date'] = temp_ship_date


df.loc[nat_order_date_mask, 'Ship Date'] = pd.NaT

print("First 10 rows after shifting 'Ship Date' to 'Order Date' for NaT entries and setting original 'Ship Date' to NaT:")
print(df[['Order Date', 'Ship Date', 'instore_purchase']].head(10))

First 10 rows after shifting 'Ship Date' to 'Order Date' for NaT entries and setting original 'Ship Date' to NaT:
  Order Date  Ship Date instore_purchase
0 2025-03-08 2025-03-28                N
1 2025-03-28        NaT                Y
2 2025-03-28        NaT                Y
3 2025-03-28        NaT                Y
4 2025-03-28        NaT                Y
5 2025-03-28        NaT                Y
6 2025-03-28        NaT                Y
7 2025-03-28        NaT                Y
8 2025-03-28        NaT                Y
9 2025-03-28        NaT                Y


In [32]:
instore_purchase_mask = df['instore_purchase'] == 'Y'
df.loc[instore_purchase_mask, 'Ship Date'] = pd.NaT

print("DataFrame Info after handling 'Ship Date' for in-store purchases:")
df.info()

DataFrame Info after handling 'Ship Date' for in-store purchases:
<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 0 to 399
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          362 non-null    int64         
 1   Customer Name     362 non-null    object        
 2   Order Date        362 non-null    datetime64[ns]
 3   Ship Date         104 non-null    datetime64[ns]
 4   Product           362 non-null    object        
 5   Quantity          362 non-null    int64         
 6   Price             362 non-null    float64       
 7   Total Price       310 non-null    float64       
 8   Region            362 non-null    object        
 9   instore_purchase  362 non-null    object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(4)
memory usage: 31.1+ KB


In [31]:
print("DataFrame Info after handling 'Order Date':")
df.info()

DataFrame Info after handling 'Order Date':
<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 0 to 399
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          362 non-null    int64         
 1   Customer Name     362 non-null    object        
 2   Order Date        362 non-null    datetime64[ns]
 3   Ship Date         104 non-null    datetime64[ns]
 4   Product           362 non-null    object        
 5   Quantity          362 non-null    int64         
 6   Price             362 non-null    float64       
 7   Total Price       310 non-null    float64       
 8   Region            362 non-null    object        
 9   instore_purchase  362 non-null    object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(4)
memory usage: 31.1+ KB


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 0 to 399
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          362 non-null    int64         
 1   Customer Name     362 non-null    object        
 2   Order Date        362 non-null    datetime64[ns]
 3   Ship Date         104 non-null    datetime64[ns]
 4   Product           362 non-null    object        
 5   Quantity          362 non-null    int64         
 6   Price             362 non-null    float64       
 7   Total Price       310 non-null    float64       
 8   Region            362 non-null    object        
 9   instore_purchase  362 non-null    object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(4)
memory usage: 31.1+ KB


In [34]:
df['ship_time'] = (df['Ship Date'] - df['Order Date']).dt.days

print("First 10 rows with 'Order Date', 'Ship Date', and 'ship_time' columns:")
print(df[['Order Date', 'Ship Date', 'ship_time']].head(10))

print("\nDataFrame Info after adding 'ship_time' column:")
df.info()

First 10 rows with 'Order Date', 'Ship Date', and 'ship_time' columns:
  Order Date  Ship Date  ship_time
0 2025-03-08 2025-03-28       20.0
1 2025-03-28        NaT        NaN
2 2025-03-28        NaT        NaN
3 2025-03-28        NaT        NaN
4 2025-03-28        NaT        NaN
5 2025-03-28        NaT        NaN
6 2025-03-28        NaT        NaN
7 2025-03-28        NaT        NaN
8 2025-03-28        NaT        NaN
9 2025-03-28        NaT        NaN

DataFrame Info after adding 'ship_time' column:
<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 0 to 399
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          362 non-null    int64         
 1   Customer Name     362 non-null    object        
 2   Order Date        362 non-null    datetime64[ns]
 3   Ship Date         104 non-null    datetime64[ns]
 4   Product           362 non-null    object        
 5   Quantity    

In [38]:
df.head(10)

,Order ID,Customer Name,Order Date,Ship Date,Product,Quantity,Price,Total Price,Region,instore_purchase,ship_time
0,2001,Chris Paul,2025-03-08,2025-03-28,Charger,3,50.0,150.0,East,N,20.0
1,2002,Eva White,2025-03-28,NaT,Tablet,2,762.5,1525.0,East,Y,NaN
2,2003,Olivia Adams,2025-03-28,NaT,Monitor,2,50.0,100.0,North,Y,NaN
3,2004,Robert Finch,2025-03-28,NaT,Mouse,2,650.0,1300.0,West,Y,NaN
4,2005,Sarah Lee,2025-03-28,NaT,Monitor,1,1895.0,1895.0,North,Y,NaN
5,2006,Olivia Adams,2025-03-28,NaT,Laptop,2,729.0,1458.0,Unknown,Y,NaN
6,2007,Mike Green,2025-03-28,NaT,Laptop,2,850.0,1700.0,South,Y,NaN
7,2008,Chris Paul,2025-03-28,NaT,Monitor,3,996.0,2988.0,South,Y,NaN
8,2009,Emma Stone,2025-03-28,NaT,Mouse,4,337.5,1350.0,North,Y,NaN
9,2010,Alan Brown,2025-03-28,NaT,Monitor,1,50.0,50.0,West,Y,NaN


In [36]:
df['Total Price'] = df['Price'] * df['Quantity']

print("Missing values in 'Total Price' after recalculation:")
print(df['Total Price'].isnull().sum())

Missing values in 'Total Price' after recalculation:
0


In [37]:
print("\nDataFrame Info:")
df.info()

print("\nDescriptive Statistics:")
print(df.describe())

print("\nMissing Values after cleaning:")
print(df.isnull().sum())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 0 to 399
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          362 non-null    int64         
 1   Customer Name     362 non-null    object        
 2   Order Date        362 non-null    datetime64[ns]
 3   Ship Date         104 non-null    datetime64[ns]
 4   Product           362 non-null    object        
 5   Quantity          362 non-null    int64         
 6   Price             362 non-null    float64       
 7   Total Price       362 non-null    float64       
 8   Region            362 non-null    object        
 9   instore_purchase  362 non-null    object        
 10  ship_time         104 non-null    float64       
dtypes: datetime64[ns](2), float64(3), int64(2), object(4)
memory usage: 33.9+ KB

Descriptive Statistics:
          Order ID                     Order Date            Ship Date  \
